In [1]:
from packages.Utility.DatasetManager import DatasetWriter

Using TensorFlow backend.


In [20]:
writer = DatasetWriter(
    path = 'dataset/images/ghim20',
    output_dir = 'dataset/bin/ghim20 227x227 5k',
    jumlah_citra_per_kelas = 250,
    fold_split = 4,
    resize = (227, 227),
    shuffle = True,
    grayscale = False
)


Scanning directory : Ayam
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Bangunan
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Bunga
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Capung
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Gunung Salju
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Kapal
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Kapal Militer
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Kembang Api
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Kuda
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Kuil
Reading: 250/250 [========================================>] 100.0%
Scanning directory : Kupu-k

In [21]:
writer.save()

Saving ...
Saved to	: dataset/bin/ghim20 227x227 5k
		  -> dataset.npy
		  -> folds.npy
		  -> labels.npy
		  -> config.json
